# Day 1: Introduction

In [ ]:
from google.cloud import bigquery

In [ ]:
# Create a "Client" object
client = bigquery.Client()

In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the "hacker_news" dataset-Tell me all of the spreadsheets in this specific dataset)
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)
# This gave us all the tables we could run queries against

In [ ]:
# Construct a reference to the "full" table (changed in video to comments)
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

In [ ]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
# We named full "table" above (changed to comments in video)
table.schema

In [ ]:
# Preview the first five lines of the "full" table
# This is like the head command in pandas or R
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Preview the first five entries in the "by" column of the "full" table (later changed to comments)
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

# Day 2: Select, From, and Where 

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset (There is only one)
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

In [ ]:
# Construct a reference to the "global_air_quality" table (i want a map that tells me how to directly 
# get to this table)
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table (Just like the head command)
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
table.schema
# This is useful as it often will contain the description of the variable

In [ ]:
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

# The triple quotations is a way to comment over multiple lines

In [ ]:
# Create a "Client" object
client = bigquery.Client()

In [ ]:
# Set up the query
query_job = client.query(query)

In [ ]:
# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

In [ ]:
us_cities

In [ ]:
# What five cities have the most measurements? (NOW WE'RE MOVING ON TO REGULAR PANDAS STUFF)
us_cities.city.value_counts().head()

In [ ]:
query = """
        SELECT city, source_name
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities.head()



In [ ]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

query_job = client.query(query)

us_cities = query_job.to_dataframe()

us_cities.head()

# Prob won't do this a lot because data on bigquery is very big and there may be limits to pull
# Bigquery caches recent queries so if you run it multiple times it'll be faster

In [ ]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = "IN" 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

# This can be helpful if it's a new dataset and you don't know how much time it will take for your query 
# to run

In [ ]:
# Only run the query if it's less than 100 MB
ONE_HUNDRED_MB = 100*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_HUNDRED_MB)

# Set up the query (will only run if it's less than 100 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

# You can run the query with a little bit of a seatbelt: We say that I only want to run jobs if they 
# are less than 100 megabytes (Remember it is only calculated in bytes)

# Remember that if it's cached and you run it again, it won't count against your bigquery quota 
# You can get an exceeded limit for bytes billed, so be careful with credit card payments obviously

# Day 3: Groupby, Having, Where

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "comments" table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

# id is the unique identifier here 
# We only look at the ones that have more than ten comments
# We're creating a text string with a valid SQL text string to it, then we hand it to the client, and 
# then the client gives it to the bigquery

In [ ]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()

# We end up with the hacker news parent posts and the number of comments for each

In [ ]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.comments`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

# You can use Count(1) if you have a column called id, but also just if you have one column with 
# unique identifiers 

In [ ]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.comments`
             GROUP BY parent
             """

In [ ]:
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """

# Day 4: Order by, Extract, Dates

Introduction
So far, you've learned how to use several SQL clauses. For instance, you know how to use SELECT to pull specific columns from a table, along with WHERE to pull rows that meet specified criteria. You also know how to use aggregate functions like COUNT(), along with GROUP BY to treat multiple rows as a single group.

Now you'll learn how to change the order of your results using the ORDER BY clause, and you'll explore a popular use case by applying ordering to dates. To illustrate what you'll learn in this tutorial, we'll work with a slightly modified version of our familiar pets table.

### ORDER BY
ORDER BY is usually the last clause in your query, and it sorts the results returned by the rest of your query.

Notice that the rows are not ordered by the ID column. We can quickly remedy this with the query below.

In [ ]:
## ORDER
# Can sort by an ID, by another column (Alphabetically for categorical for example)
# To reverse orders, use DESC at the end

# query = '''
# SELECT ID Name, Animal
# FROM 'thequery'
# ORDER BY Animal DESC
# '''

### Dates
Next, we'll talk about dates, because they come up very frequently in real-world databases. There are two ways that dates can be stored in BigQuery: as a DATE or as a DATETIME.

In [ ]:
## The DATE format has the year first, then the month, and then the day. It looks like this:

## YYYY-[M]M-[D]D

# YYYY: Four-digit year
# [M]M: One or two digit month
# [D]D: One or two digit day

# The DATETIME format is like the date format ... but with time added at the end.

# Can look up standard-sql/date_functions on big query if needed


### EXTRACT
Often you'll want to look at part of a date, like the year or the day. You can do this with EXTRACT. We'll illustrate this with a slightly different table, called pets_with_date.**

In [ ]:
# query = """
# SELECT Name, EXTRACT(DAY from Date) AS Day
# FROM 'queryname'
# """

# query = """
# SELECT Name, EXTRACT(WEEK from Date) AS Week
# FROM 'queryname'
# """

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "nhtsa_traffic_fatalities" dataset
dataset_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "accident_2015" table
table_ref = dataset_ref.table("accident_2015")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "accident_2015" table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

# This query selects the count of consecutive number, and extract day of week from time stamp of crash
# Note that it seems that we're referencing the database and the table in one go in from.

In [ ]:
# We won't run this query if it is going to take more than this amount of billing!
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)
# We set up the query above, so all we needed to change was the text in query, which we just did
# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

# Sunday is 1, but most accidents are on Saturday (AM drunk driving probs)
# Safest day of the week looks like Tuesday

# Day 5: AS and WITH

### Introduction
With all that you've learned, your SQL queries are getting pretty long, which can make them hard understand (and debug).

You are about to learn how to use AS and WITH to tidy up your queries and make them easier to read.

Along the way, we'll use the familiar pets table, but now it includes the ages of the animals.

### AS
You learned in an earlier tutorial how to use AS to rename the columns generated by your queries, which is also known as aliasing. This is similar to how Python uses as for aliasing when doing imports like import pandas as pd or import seaborn as sns.

To use AS in SQL, insert it right after the column you select. Here's an example of a query without an AS clause

In [ ]:
# Example Without AS
### query ="""
# SELECT animal, COUNT(ID)
# FROM 'bigqueryname'
# GROUP BY Animal
# """

# Example With AS
### query ="""
# SELECT animal, COUNT(ID) AS Number 
# FROM 'bigqueryname'
# GROUP BY Animal
# 

# These queries return the same information, but in the second query the column returned by the 
# COUNT() function will be called Number, rather than the default name of f0__

### WITH ... AS
On its own, AS is a convenient way to clean up the data returned by your query. It's even more powerful when combined with WITH in what's called a "common table expression".

A common table expression (or CTE) is a temporary table that you return within your query. CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them.

For instance, you might want to use the pets table to ask questions about older animals in particular. So you can start by creating a CTE which only contains information about animals more than five years old like this:

In [ ]:
### query ="""
# WITH Seniors AS
# (SELECT ID, Name
# FROM 'bigqueryname'
# WHERE Years_old > 5
# )
# """

# Note that the above is just a part of a query and it is not complete

While this incomplete query above won't return anything, it creates a CTE that we can then refer to (as Seniors) while writing the rest of the query.

We can finish the query by pulling the information that we want from the CTE. The complete query below first creates the CTE, and then returns all of the IDs from it.

In [ ]:
### query ="""
# WITH Seniors AS
# (SELECT ID, Name
# FROM 'bigqueryname'
# WHERE Years_old > 5
# )
# SELECT ID 
# FROM Seniors

# This is the full version because now we're actually doing something

You could do this without a CTE, but if this were the first part of a very long query, removing the CTE would make it much harder to follow.

Also, it's important to note that CTEs only exist inside the query where you create them, and you can't reference them in later queries. So, any query that uses a CTE is always broken into two parts: (1) first, we create the CTE, and then (2) we write a query that uses the CTE.

### Example: How many Bitcoin transactions are made per month?
We're going to use a CTE to find out how many Bitcoin transactions were made each day for the entire timespan of a bitcoin transaction dataset.

We'll investigate the transactions table. Here is a view of the first few rows. (The corresponding code is hidden, but you can un-hide it by clicking on the "

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "crypto_bitcoin" dataset
dataset_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "transactions" table
table_ref = dataset_ref.table("transactions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "transactions" table
client.list_rows(table, max_results=5).to_dataframe()

Since the block_timestamp column contains the date of each transaction in DATETIME format, we'll convert these into DATE format using the DATE() command.

We do that using a CTE, and then the next part of the query counts the number of transactions for each date and sorts the table so that earlier dates appear first.

In [ ]:
# Query to select the number of transactions per date, sorted by date
query_with_CTE = """ 
                 WITH time AS 
                 (
                     SELECT DATE(block_timestamp) AS trans_date
                     FROM `bigquery-public-data.crypto_bitcoin.transactions`
                 )
                 SELECT COUNT(1) AS transactions,
                        trans_date
                 FROM time
                 GROUP BY trans_date
                 ORDER BY trans_date
                 """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_with_CTE, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
transactions_by_date = query_job.to_dataframe()

# Print the first five rows
transactions_by_date.head()

# We could technically create more thana one of these CTEs in the same query if we'd want to.
# The idea is that we could then do joins internally

Since they're returned sorted, we can easily plot the raw results to show us the number of Bitcoin transactions per day over the whole timespan of this dataset.

In [ ]:
transactions_by_date.set_index('trans_date').plot()

# The set index part means that we are sorting by transaction date

As you can see, common table expressions (CTEs) let you shift a lot of your data cleaning into SQL. That's an especially good thing in the case of BigQuery, because it is vastly faster than doing the work in Pandas

# Day 6: Joining Data

### Introduction
You have the tools to obtain data from a single table in whatever format you want it. But what if the data you want is spread across multiple tables?

That's where JOIN comes in! JOIN is incredibly important in practical SQL workflows. So let's get started.

### Example
We'll use our imaginary pets table, which has three columns:

ID - ID number for the pet
Name - name of the pet
Animal - type of animal
We'll also add another table, called owners. This table also has three columns:

ID - ID number for the owner (different from the ID number for the pet)
Name - name of the owner
Pet_ID - ID number for the pet that belongs to the owner (which matches the ID number for the pet in the pets table)

For example,

the pets table shows that Dr. Harris Bonkers is the pet with ID 1.
The owners table shows that Aubrey Little is the owner of the pet with ID 1.
Putting these two facts together, Dr. Harris Bonkers is owned by Aubrey Little.

Fortunately, we don't have to do this by hand to figure out which owner goes with which pet. In the next section, you'll learn how to use JOIN to create a new table combining information from the pets and owners tables.

### JOIN
Using JOIN, we can write a query to create a table with just two columns: the name of the pet and the name of the owner.

In [ ]:
# query ="""
# SELECT p.name AS Pet_Name, 
# o.Name AS Owner_Name
# FROM `bigqueryname.pets` AS p
#INNER JOIN `bigqueryname.owners` AS o
#   ON p.ID = o.Pet_ID
# """

# We alias down, but can reference the alias above like we do here. 
# Name is a column in the pets table 
# Inner Join means we only get items that occur in both
    # Logical OR -> You'll get it if you get it in both of them and not just one
    # Logical AND -> Get everything from both tables (Outer Join)
        # Don't do it. You'll blow through your quota if you don't have the safety belt
    # A guy without a pet would not end up in the merge then
    # Inner Join is the safest when it comes to how big the resulting dataset is
# ON tells us which columns to look for to link the two datasets

### Example: How many files are covered by each type of software license?
GitHub is the most popular place to collaborate on software projects. A GitHub repository (or repo) is a collection of files associated with a specific project.

Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on how they are used. For our example, we're going to look at how many different files have been released under each license.

We'll work with two tables in the database. The first table is the licenses table, which provides the name of each GitHub repo (in the repo_name column) and its corresponding license. Here's a view of the first five rows.

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results=5).to_dataframe()

The second table is the sample_files table, which provides, among other information, the GitHub repo that each file belongs to (in the repo_name column). The first several rows of this table are printed below

In [ ]:
# Construct a reference to the "sample_files" table
files_ref = dataset_ref.table("sample_files")

# API request - fetch the table
files_table = client.get_table(files_ref)

# Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results=5).to_dataframe()

Next, we write a query that uses information in both tables to determine how many files are released in each license.

In [ ]:
# Query to determine the number of files per license, sorted by number of files
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """
# I can change this to COUNT(sf.id) if I want to do the counting from a specific dataset

# What are we doing here? 
    # We select license from the table we define as L, which is the licenses table in the github_repos
    # dataset
    # number_of_files will technically come from sample-files
    # We do an inner join then

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

In [ ]:
# Print the DataFrame
file_count_by_license

# Day 7: Machine Learning with SQL

This tutorial is an introduction to BigQuery ML. It is based on the official documentation tutorial. In this tutorial, you use the sample Google Analytics sample dataset for BigQuery to create a model that predicts whether a website visitor will make a transaction. For information on the schema of the Analytics dataset, see BigQuery export schema.

There is also an accopmnying livestream video, which can be found here and a notebook with exercises for you to try out.

###What is BigQuery ML and when should you use it?
BigQuery Machine Learning (BQML) is a toolset that allows you to train and serve machine learning models directly in BigQuery. This has several advantages:

You don't have to read your data into local memory. One question I get a lot is "how can I train my ML model if my dataset is just too big to fit on my computer?". You can subsample your dataset, of course, but you can also use tools like BQML that train your model directly in your database.

You don't have to use multiple languages. Particularly if you're working in a team where most of your teammates don't know Python or R or your preferred language for modelling, working in SQL can make it easier for you to collaborate.

You can serve your model immediately after it's trained. Because your model is already in the same place as your data, you can make predictions directly from your database. This lets you get around the hassle of cleaning up your code and either putting it intro production or passing it off to your engineering colleagues.

BQML probably won't replace all your modelling tools, but it's a nice quick way to train and serve a model without spending a lot of time moving code or data around.

Objectives
In this tutorial, you will use:

BQML to create a binary logistic regression model using the CREATE MODEL statement
The ML.EVALUATE function to evaluate the ML model
The ML.PREDICT function to make predictions using the ML model[](http://)

### Step one: Setup and create your dataset
First, you'll need to create a BigQuery dataset to store your ML model. You don't actually have to upload any data; the only table in it will be the one with your trained model.

If you're curious about the client and what it does, check out the first lesson in the Intro SQL course and the accompying video.)

In [ ]:
# Set your own project id here
PROJECT_ID = 'kagglebq-test'

# Create a client instance for your project
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")

In [ ]:
# Create a small array and save it as a csv
import numpy
a = numpy.asarray([[1,2,3], [4,5,6], [7,8,9]])
numpy.savetxt("foo.csv", a, delimiter=",")

In [ ]:
# Create a new dataset 
client.create_dataset('new_dataset')

In [ ]:
# Create a new table in that dataset ()
client.create_table(f"kagglebq-test.new_dataset.new_table")

In [ ]:
# Upload CSV file to BigQuery 
# foo.csv is currently in our directory
filename = "foo.csv"
dataset_id = "new_dataset"
table_id = "new_table"

# Tell the client everything it needs to know to upload our csv
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.autodetect = True

# Load the CSV into BigQuery
with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result() #Waits for table load to complete

# Results:
print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

# It's 3 because we made an array with 3 rows

In [ ]:
#query
query = f""" SELECT *
        FROM `kagglebq-test.new_dataset.new_table`"""

#Set up the query 
query_job = client.query(query)

#API request -run the query, and return a pandas DataFrame
data = query_job.to_dataframe()

# Remember that tricky business with the back ticks!

In [ ]:
data

#### The above was for the first set up video. The below is for the actual lesson.

In [ ]:
# Set your own project id here
PROJECT_ID = 'kagglebq-test'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('bqml_tutorial', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

# Previously, we interacted directly with the client. 
# Today, we're using the notebook magic.
    # You can type sql directly without putting in triple quotes, send to query object, send that to client 
    # Will only work in jupyter
    # Less flexible but less typing 

For this example, we will use the Google Analytics sample dataset to predict whether a website visitor will make a transaction. First, let's take a quick look at the data we'll be using.

In [ ]:
# create a reference to our table
table = client.get_table("bigquery-public-data.google_analytics_sample.ga_sessions_*")

# look at five rows from our dataset
client.list_rows(table, max_results=5).to_dataframe()

# Google analytics is the dataset and ga.sessions_* is a range of tables
    # Every table that starts with ga_sessions_
    # Each day gets a new table in this dataset, so we're getting the table for all days
    
# Strux or recrods: JSON file inside a cell 
    # If we look at the schema of the table, we can see the entire defined schema

This table looks a little different from some of the ones you've seen in the Intro to SQL course. Notably the columns for "totals", "trafficSource", "device", "geoNetwork", "customDimensions" and "hits" are of a new data type called STRUCT (or RECORD) which is used for nested data structures. We'll learn more about these in the advanced SQL course, but for now all you need to know is:

Each cell in a specific column has the same fields in it. (These are specified in the schema.)
You can get the information from a specific field for a whole column using the syntax COLUMN_NAME.FIELD_NAME.
So for the "totals" column, we could get the "transactions" field by using the syntax totals.transactions. (You can see all the fields in a record from the "totals" column below).

In [ ]:
# create a small sample dataframe
sample_table = client.list_rows(table, max_results=5).to_dataframe()

# get the first cell in the "totals" column
sample_table.totals[0]

# Column name . the field name we are interested in.
    # We created a small dataset from the table, and then we looked in the totals column, and got info from first row 

### Step two: Create your model
Next, we will create a logistic regression model for classification.

The standard SQL query uses a CREATE MODEL statement to create and train the model. You can find the documentation for this fuction here.

The BigQuery Python client library provides a custom magic command so that you don't need to set up the queries yourself. To load the magic commands from the client library, run the following code.

If you prefer not to use the magic command, you can use client-based way of writing queries covered in the Intro to SQL course. They'll both work the same, it's just a slightly different style of writing code. :)

For this problem, we'll be trying to predict transactions from the totals column, so keep this in mind! :)

In [ ]:
%load_ext google.cloud.bigquery

%load_ext is one of the many Jupyter built-in magic commands. See the Jupyter documentation for more information about %load_ext and other magic commands.

The BigQuery client library provides a cell magic, %%bigquery, which runs a SQL query and returns the results as a Pandas DataFrame. Once you use this command the rest of your cell will be treated as a SQL command. (Note that tab complete won't work for SQL code written in this way.)

Here's the the query that will train our model:

In [ ]:
%%bigquery
CREATE MODEL IF NOT EXISTS `bqml_tutorial.sample_model2`
OPTIONS(model_type='logistic_reg') AS
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'
    
# There is apparently no overwriting, so you can just run this once unless you delete it. 
# Otherwise you just have to rename (that's why it's now sample_model2.

In [ ]:
# REFERENCING THE CELL ABOVE
# First thing we want to do is create our model. 
# If you look in the console on GCP, you'll now see the sample_model. 
# We use Options to say we're training a logistic regression 
    # You could also choose to say which column you want to be your target column (we use the first column here and 
    #call it label - could of specified earlier too)
    # You can choose your optimization strategy, but a lot is done automatically
    # You can choose testing training split, but that's done automatically too
# Then we have actual variables we're using as independent. 
    # If you try to predict a null, you'll get an error obviously, so we had to invert nones into 0s or otherwise 1
    # We're predicting whether or not there will be a transaction, but the number of transactions 
# We use the ifnull to clean essentially - not from country none, but the country is unknown
    
# At the bottom, we say what tables we'll use to train the model 
# Then we say which of the tables we'll actually use 


# **Tip:** Using ```CREATE OR REPLACE MODEL``` rather than just ```CREATE MODEL``` ensures you don't get an error if you 
# want to run this command again without first deleting the model you've created.

Let's break down this command a little bit. This is a fairly fancy query, which is nice because it lets you see a lot of the options you have when using BQML.

This line is writing our model to BigQuery. (The fact that BQML requires write permissions is why you needed to set up your GCP account; the default Kaggle connection doesn't allow you write tables, only query them.

In [ ]:
%%bigquery
CREATE MODEL IF NOT EXISTS `bqml_tutorial.sample_model2`

Here we're specifying that our model will be logistic regression, so we know it's a classification task. Next we need to actually define our model.

In [ ]:
OPTIONS(model_type='logistic_reg') AS

The code under the SELECT clause is where we define the variable we're trying to predict as well as what variables we want to use to predict it.

The column we alias as "label" will be our dependent variable, the thing we're trying to predict. The IF(totals.transactions IS NULL, 0, 1) bit just means that if the values of the transactions field is NULL or None, we'll assign it a value of "0", otherwise it will have a value of one. Since we have two labels, BQML will automatically use binary regression for us.

The other four rows say what information we want to use to predict that label. Here we're information on the device operating system, whether it's a mobile device, the country and the number of pageviews. For the os, country and pageviews, we're assigning appropriate values to null fields. Either making them blank (so we're not saying people are from NULL country!) or setting them to 0.

In [ ]:
SELECT
  IF(totals.transactions IS NULL, 0, 1) AS label,
  IFNULL(device.operatingSystem, "") AS os,
  device.isMobile AS is_mobile,
  IFNULL(geoNetwork.country, "") AS country,
  IFNULL(totals.pageviews, 0) AS pageviews


The FROM clause specifies the table (in this case more than one table) that we're going to get our data from.

In [ ]:
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`

And finally the WHERE clause specifies the range of tables to use to train our model. In this case it looks like a new table is being produced every day, so we'll be using data from between August 1, 2016 and June 30, 2017 to train our model. (You would only need to specify range if you were using some tables out of all the possible tables in your dataset.)

In [ ]:
WHERE
  _TABLE_SUFFIX BETWEEN '20160801' AND '20170630'

The query takes several minutes to complete. After the first iteration is complete, your model (sample_model) appears in the navigation panel of the BigQuery UI. Because the query uses a CREATE MODEL statement to create a table, you do not see query results. The output is an empty string.

### Step three: Get training statistics
To see the results of the model training, you can use the ML.TRAINING_INFO function, or you can view the statistics in the BigQuery UI. In this tutorial, you use the ML.TRAINING_INFO function.

A machine learning algorithm builds a model by examining many examples and attempting to find a model that minimizes loss. This process is called empirical risk minimization.

Loss is the penalty for a bad prediction — a number indicating how bad the model's prediction was on a single example. If the model's prediction is perfect, the loss is zero; otherwise, the loss is greater. The goal of training a model is to find a set of weights that have low loss, on average, across all examples.

To see the model training statistics that were generated when you ran the CREATE MODEL query, run the following:

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_tutorial.sample_model`)
ORDER BY iteration 

# Because we had set it up in previous iteration (changes to samplemodel2 after to run again)

Note: Typically, it is not a best practice to use a SELECT * query. Because the model output is a small table, this query does not process a large amount of data. As a result, the cost is minimal.

The loss column represents the loss metric calculated after the given iteration on the training dataset. Since you performed a logistic regression, this column is the log loss. The eval_loss column is the same loss metric calculated on the holdout dataset (data that is held back from training to validate the model).

At this point you'll notice that BQML has taken care of some of the common ML decisions for you:

Splitting into training & evaluation datasets to help detect overfitting
Early stopping (stopping training when additional iterations would not improve performance on the evaluation set)
Picking and updating learning rates (starting with a low learning rate and increasing it over time)
Picking an optimization strategy (batch gradient descent for large datasets with high cardinality, normal equation for small datasets where it would be faster)
For more details on the ML.TRAINING_INFO function, see the BQML syntax reference.

### Step four: Evaluate your model
After creating your model, you evaluate the performance of the classifier using the ML.EVALUATE function.

A classifier is one of a set of enumerated target values for a label. For example, in this tutorial you are using a binary classification model that detects transactions. The two classes are the values in the label column: 0 (no transactions) and not 1 (transaction made).

To run the ML.EVALUATE query that evaluates the model, run the following:

In [ ]:
%%bigquery
SELECT
  *
FROM ML.EVALUATE(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IF(totals.transactions IS NULL, 0, 1) AS label,
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(geoNetwork.country, "") AS country,
    IFNULL(totals.pageviews, 0) AS pageviews
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))

While it's helpful to see these metrics, it's also common to plot the ROC curve when evaluating model performance for binary logistic regression. We can do this by using the ML.ROC_CURVE() function.

Pro-tip: You can save the output of a bigquery magic cell by putting a variable name to the right of the %%bigquery command. Here I've saved the output of the next cell as the variable "roc" .

In [ ]:
%%bigquery roc
SELECT
  *
FROM
  ML.ROC_CURVE(MODEL `bqml_tutorial.sample_model`)

In [ ]:
# check out the data that was returned...
roc.head()

# Super useful.

In [ ]:
# and plot our ROC curve!
import matplotlib.pyplot as plt

# plot the false positive rate by true postive rate (aka recall)
plt.plot(roc.false_positive_rate, roc.recall)

### Step five: Use your model to predict outcomes
Now that you have evaluated your model, the next step is to use it to predict outcomes. You use your model to predict the number of transactions made by website visitors from each country. And you use it to predict purchases per user.

To run the query that uses the model to predict the number of transactions by country. (Note that you only need to SELECT the column you're grouping by, not every column you used in your prediction.)

In [ ]:
%%bigquery
SELECT
  country,
  SUM(predicted_label) as total_predicted_purchases
FROM ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(totals.pageviews, 0) AS pageviews,
    IFNULL(geoNetwork.country, "") AS country
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
  GROUP BY country
  ORDER BY total_predicted_purchases DESC
  LIMIT 10

In [ ]:
# We typically end up just repeating the code upon prediction 
# Above, I select country and sum over the predicted labels 
    # I group by country 
    # I order by predicted purchases descending 
    # This is the number of visits that I anticipate will become a transaction
    # Then I only show the first ten rows

In the next example, you try to predict the number of transactions each website visitor will make. This query is identical to the previous query except for the GROUP BY clause. Here the GROUP BY clause — GROUP BY fullVisitorId — is used to group the results by visitor ID.

To run the query that predicts purchases per user:

In [ ]:
%%bigquery
SELECT
  fullVisitorId,
  SUM(predicted_label) as total_predicted_purchases
FROM ML.PREDICT(MODEL `bqml_tutorial.sample_model`, (
  SELECT
    IFNULL(device.operatingSystem, "") AS os,
    device.isMobile AS is_mobile,
    IFNULL(totals.pageviews, 0) AS pageviews,
    IFNULL(geoNetwork.country, "") AS country,
    fullVisitorId
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170801'))
  GROUP BY fullVisitorId
  ORDER BY total_predicted_purchases DESC
  LIMIT 10

In [ ]:
# Above is the people I think are likely to make purchases
# If we were doing B2B, this might be the business that I want to reach out to obviously as they are likely to make a 
# purchase

Cleanup
To avoid cluttering up your Google Cloud Platform account with the models you trained in this tutorial:

You can delete the project you created.
Or you can keep the project and delete the dataset.

# Day 8: Analytics Functions

#### Introduction
In the Intro to SQL micro-course, you learned how to use aggregate functions, which perform calculations based on sets of rows. In this tutorial, you'll learn how to define analytic functions, which also operate on a set of rows. However, unlike aggregate functions, analytic functions return a (potentially different) value for each row in the original table.

We'll work with the San Francisco Open Data dataset. We begin by reviewing the first several rows of the bikeshare_trips table. (The corresponding code is hidden, but you can un-hide it by clicking on the "Code" button below.)

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "san_francisco" dataset
dataset_ref = client.dataset("san_francisco", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "bikeshare_trips" table
table_ref = dataset_ref.table("bikeshare_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Each row of the table corresponds to a different bike trip, and we can use an analytic function to calculate the cumulative number of trips for each date in 2015

In [ ]:
# Query to count the (cumulative) number of trips per day
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

# First thing we do is a common table expression (Creating a new table we can refer to in rest of query called trips_by_day)
    # Select date from start_date, counting all rows as num_trips and group_by trip_date 
    # All rows from same date together and count how many per date 
    # We're only looking at 2015
# We want cumulative sum of bike trips 
    # We use a windowing function
        # Not doinng a partition here 
        # We order by trip date
        # Unbounded preceding and current row means all of previous rows and the current row
            # We take the sum of all that and call it cumulative_trip
            # All comes from our CTE


# Run the query, and return a pandas DataFrame
num_trips_result = client.query(num_trips_query).result().to_dataframe()
num_trips_result.head()

The next query tracks the stations where each bike began (in start_station_id) and ended (in end_station_id) the day on October 25, 2015.

In [ ]:
# Query to track beginning and ending stations on October 25, 2015, for each bike
start_end_query = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """
# We create two windowing functions
    # Find first value of start station ID over the windowing function where we partition by bike number, order by start_date,
    # and then look at all rows in partition
    
    # Then last value of end station ID, partition by bike number, order by start, but look at last value of start 
    # station id 
    # We ignore start and end for trips that were not the first trip on day when it is 2015/10/25

# Run the query, and return a pandas DataFrame
start_end_result = client.query(start_end_query).result().to_dataframe()
start_end_result.head()

# Day 9: Joins and Unions

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("comments")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

In [ ]:
# Construct a reference to the "stories" table
table_ref = dataset_ref.table("stories")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

The query below pulls information from the stories and comments tables to create a table showing all stories posted on January 1, 2012, along with the corresponding number of comments. We use a LEFT JOIN so that the results include stories that didn't receive any comments.

In [ ]:
# Query to select all stories posted on January 1, 2012, with number of comments
join_query = """
             WITH c AS
             (
             SELECT parent, COUNT(*) as num_comments
             FROM `bigquery-public-data.hacker_news.comments` 
             GROUP BY parent
             )
             SELECT s.id as story_id, s.by, s.title, c.num_comments
             FROM `bigquery-public-data.hacker_news.stories` AS s
             LEFT JOIN c
             ON s.id = c.parent
             WHERE EXTRACT(DATE FROM s.time_ts) = '2012-01-01'
             ORDER BY c.num_comments DESC
             """
# So what are we doing with this query?
    # We'll have two columns in our CTE, parents and num_comments and it's grouped by parents 
    # We then use c in our join query as the alias for the CTE created 
    # Remember that SQL queries aren't evaluate from top to bottom so can call out s. or c. before they are alias created
    # The table that comes right after the JOIN call is the one that you are joining to (c)
    # We filter by extracting date from time stamp and then we order 

# Run the query, and return a pandas DataFrame
join_result = client.query(join_query).result().to_dataframe()
join_result.head()

In [ ]:
# None of these stories received any comments
join_result.tail()

Next, we write a query to select all usernames corresponding to users who wrote stories or comments on January 1, 2014. We use UNION DISTINCT (instead of UNION ALL) to ensure that each user appears in the table at most once.

In [ ]:
# Query to select all users who posted stories or comments on January 1, 2014
union_query = """
              SELECT c.by
              FROM `bigquery-public-data.hacker_news.comments` AS c
              WHERE EXTRACT(DATE FROM c.time_ts) = '2014-01-01'
              UNION DISTINCT
              SELECT s.by
              FROM `bigquery-public-data.hacker_news.stories` AS s
              WHERE EXTRACT(DATE FROM s.time_ts) = '2014-01-01'
              """

# DISTINCT says I don't want any repeats and only unique values 
    # If looking at census data from different countries and you want to find out most common first name in world, 
    # you wouldn't want distinct
    # We only care about the username here though, so DISTINCT makes sense. 
    # So we can filter and append at same time. 

# Run the query, and return a pandas DataFrame
union_result = client.query(union_query).result().to_dataframe()
union_result.head()

To get the number of users who posted on January 1, 2014, we need only take the length of the DataFrame.

In [ ]:
# Number of users who posted stories or comments on January 1, 2014
len(union_result)

# Day 10: Nested and Repeated Data

How do we interact with hierarchical data? To query a column with nested data, we need to identify each field in the context of the column that contains it. So we just call it out with a dot. 

How do we interact with repeated data?  In this case, to collapse this information into a single table, we need to leverage a different datatype. We say that the "Toys" column contains repeated data, because it permits more than one value for each row. Each entry in a repeated field is an ARRAY, or an ordered list of (zero or more) values with the same datatype. When querying repeated data, we need to put the name of the column containing the repeated data inside an UNNEST() function. Keep in mind that we say the column name and then after from specify we want it unnested as that name we already called out. This essentially flattens the repeated data (which is then appended to the right side of the table) so that we have one element on each row.

What about nested and repeated data at the same time? We essentially use our unnest as an alias and then use the alias. notation. Since the "Toys" column is repeated, we flatten it with the UNNEST() function. And, since we give the flattened column an alias of t, we can refer to the "Name" and "Type" fields in the "Toys" column as t.Name and t.Type, respectively.

We'll work with the Google Analytics Sample dataset. It contains information tracking the behavior of visitors to the Google Merchandise store, an e-commerce website that sells Google branded items.

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "google_analytics_sample" dataset
dataset_ref = client.dataset("google_analytics_sample", project="bigquery-public-data")

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("ga_sessions_20170801")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

The table has many nested fields, which you can verify by looking at either the data dictionary (hint: search for appearances of 'RECORD' on the page) or the table preview above.

In our first query against this table, we'll work with the "totals" and "device" columns.

In [ ]:
print("SCHEMA field for the 'totals' column:\n")
print(table.schema[5])
# We are looking at totals

print("\nSCHEMA field for the 'device' column:\n")
print(table.schema[7])
# We are looking at device

# Repeated data has square brackets around it. 

We refer to the "browser" field (which is nested in the "device" column) and the "transactions" field (which is nested inside the "totals" column) as device.browser and totals.transactions in the query below:

In [ ]:
# Query to count the number of transactions per browser
query = """
        SELECT device.browser AS device_browser,
            SUM(totals.transactions) as total_transactions
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
        GROUP BY device_browser
        ORDER BY total_transactions DESC
        """
# We're looking in the device column and then getting information from the browser field and then looking in the totals 
# columns and getting the transactions

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()

Now we'll work with the "hits" column as an example of data that is both nested and repeated. Since:

"hits" is a STRUCT (contains nested data) and is repeated,
"hitNumber", "page", and "type" are all nested inside the "hits" column, and
"pagePath" is nested inside the "page" field,
we can query these fields with the following syntax:

In [ ]:
# Query to determine most popular landing point on the website
query = """
        SELECT hits.page.pagePath as path,
            COUNT(hits.page.pagePath) as counts
        FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`, 
            UNNEST(hits) as hits
        WHERE hits.type="PAGE" and hits.hitNumber=1
        GROUP BY path
        ORDER BY counts DESC
        """

# Hits is a record so we know it's a nested data structure. There are square brackets, so we know it's repeated too. 
# The data type of page is a record too. We have a record inside another record. page is nullable, so it's not repeated
# Inside page, we have pagePath

# We're unnesting hits, then filtering results with where, then group by path, and order by counts 

# Run the query, and return a pandas DataFrame
result = client.query(query).result().to_dataframe()
result.head()